In [3]:
jupyter_pwd = %pwd
if jupyter_pwd == "/":
    %cd /workspace


In [4]:
# ipynb形式のライブラリのインポート
%run ./lib/lib.ipynb

# 生データの入ったCSVファイルの保持されたディレクトリ名を格納している変数
csvDirPath = "./csv_files/"

# NPBのベンチマーク名のリスト
benchmarkNames = ["cg", "ep", "ft", "is", "lu", "mg"]

# LULESH ベンチマークプログラムのプロセス数・問題サイズ・イテレーション数
lulesh_processes: list[int] = [8, 27, 64, 125, 216, 343, 512]
lulesh_iterations: list[int] = [8, 16, 32, 64, 128, 256]
lulesh_sizes: list[int] = [16, 24, 32, 48, 64, 128]

DEBUG:__main__:hello


In [5]:
# ipynb形式のライブラリノートを.py形式に変更したものをインポート
import lib
import lib.lab_lib
from lib.lab_lib import *

DEBUG:lib.lab_lib:hello


# TODO

1. ✅CG, MGで時間をExtra-Pから取得できることを確認
2. ✅時間の単位を「秒」にする
    * 列「Inclusive total msec」に一部存在している”：”で区切られた値の単位は、msec ではなく分と秒を区切る”：”
3. ✅相対コストの算出関数の実装
4. ✅Extra-P のDocker環境の構築
4. Extra-P による予測との組み合わせを行う
    * なにをどう組み合わせるのかがわかっていないので、それは確かめる
    * 元データ, Extra-P単体で予測したデータ, <何か> で結果を作成

# 予測結果に必要なもの

* ✅元データ（予測環境の生データ）
* ✅Extra-P単体で予測したデータ
* Extra-Pで各関数の実行時間を予測し、それにコール回数を掛けた値で予測したデータ
    * ✅コール回数で実行時間を割る
    * ✅Extra-Pでモデルを作成 <- コール回数で実行時間を割る
    * ✅👆を利用して予測
    * ✅本プログラムでモデルを作成 <- コール回数で実行時間を割る
    * ✅👆を利用して予測
    * 「コール回数 * 実行時間」を計算して予測

上記のセルよりモデルは次式

$$ 2.5521930556315375 + 0.013737041505459383 * iteration + 3.41165137739477 * 10^7 * iteration * size^3 * \log_2 size $$

In [6]:
train_lulesh_processes: list[int] = [8, 27, 64, 125, 216, 343]
train_lulesh_iterations: list[int] = [8, 16, 32, 64, 128]
train_lulesh_sizes: list[int] = [16, 24, 32, 48, 64]

test_lulesh_processes: list[int] = [512]
test_lulesh_iterations: list[int] = [256]
test_lulesh_sizes: list[int] = [128]

In [7]:
trainDF_lulesh: pd.DataFrame = return_rawDF_lulesh(
    list_process=train_lulesh_processes,
    list_iteration=train_lulesh_iterations,
    list_size=train_lulesh_sizes,
    csvDir=csvDirPath,
)

# InclusivePerCall列の生成
trainDF_lulesh["InclusivePerCall"] = -1
trainDF_lulesh = trainDF_lulesh.reset_index()
for i, sr in trainDF_lulesh.iterrows():
    trainDF_lulesh.at[i, "InclusivePerCall"] = convertPprofTime(
        sr["Inclusive"]
    ) / float(sr["#Call"])
trainDF_lulesh = trainDF_lulesh.sort_values("#Call")
# Inclusive列の整形
_before_converted: list[float] = list(trainDF_lulesh["Inclusive"])
_after_converted: list[float] = map(convertPprofTime, _before_converted)
trainDF_lulesh["Inclusive"] = list(_after_converted)

trainDF_lulesh

,index,%Time,Exclusive,Inclusive,#Call,#Subrs,Name,process,iteration,size,InclusivePerCall
4799,31,0.0,0.000283,2.830000e-07,2.915450e-03,0.0,void_VerifyAndWriteFinalOutput(Real_t_Domain,343,128,64,9.706906e-05
4671,31,0.0,0.000283,2.830000e-07,2.915450e-03,0.0,void_VerifyAndWriteFinalOutput(Real_t_Domain,343,128,16,9.706906e-05
4031,31,0.0,0.000245,2.450000e-07,2.915450e-03,0.0,void_VerifyAndWriteFinalOutput(Real_t_Domain,343,8,16,8.403505e-05
4063,31,0.0,0.000259,2.590000e-07,2.915450e-03,0.0,void_VerifyAndWriteFinalOutput(Real_t_Domain,343,8,24,8.883706e-05
4095,31,0.0,0.000245,2.450000e-07,2.915450e-03,0.0,void_VerifyAndWriteFinalOutput(Real_t_Domain,343,8,32,8.403505e-05
...,...,...,...,...,...,...,...,...,...,...,...
772,4,9.3,"4,943",4.943000e+00,3.381660e+07,0.0,Real_t_CalcElemVolume(const,8,128,64,1.461708e-07
3176,8,5.6,"7,406",7.406000e+00,3.381660e+07,0.0,Real_t_CalcElemVolume(const,125,128,64,2.190049e-07
3972,4,7.6,"4,979",4.979000e+00,3.381660e+07,0.0,Real_t_CalcElemVolume(const,216,128,64,1.472354e-07
4774,6,7.5,"4,982",4.982000e+00,3.381660e+07,0.0,Real_t_CalcElemVolume(const,343,128,64,1.473241e-07


In [8]:
testDF_lulesh: pd.DataFrame = return_rawDF_lulesh(
    list_process=test_lulesh_processes,
    list_iteration=test_lulesh_iterations,
    list_size=test_lulesh_sizes,
    csvDir=csvDirPath,
)

# InclusivePerCall列の生成
testDF_lulesh["InclusivePerCall"] = -1
testDF_lulesh = testDF_lulesh.reset_index()
for i, sr in testDF_lulesh.iterrows():
    testDF_lulesh.at[i, "InclusivePerCall"] = convertPprofTime(sr["Inclusive"]) / float(
        sr["#Call"]
    )
testDF_lulesh = testDF_lulesh.sort_values("#Call")

# Inclusive列の整形
_before_converted: list[float] = list(testDF_lulesh["Inclusive"])
_after_converted: list[float] = map(convertPprofTime, _before_converted)
testDF_lulesh["Inclusive"] = list(_after_converted)

testDF_lulesh

,index,%Time,Exclusive,Inclusive,#Call,#Subrs,Name,process,iteration,size,InclusivePerCall
31,31,0.0,0.000256,2.560000e-07,1.953120e-03,0.0,void_VerifyAndWriteFinalOutput(Real_t_Domain,512,256,128,1.310723e-04
28,28,0.0,0.00152,3.580000e-06,1.000000e+00,2.0,void_ParseCommandLineOptions(int_char_**_Int_t...,512,256,128,3.580000e-06
27,27,0.0,0.0122,1.220000e-05,1.000000e+00,0.0,void_InitMeshDecomp(Int_t_Int_t_Int_t,512,256,128,1.220000e-05
26,26,0.0,0.0299,2.990000e-05,1.000000e+00,0.0,void_Domain::SetupSymmetryPlanes(Int_t),512,256,128,2.990000e-05
25,25,0.0,1,1.000000e-03,1.000000e+00,0.0,void_Domain::~Domain(),512,256,128,1.000000e-03
23,23,0.0,4,4.000000e-03,1.000000e+00,0.0,void_Domain::SetupCommBuffers(Int_t),512,256,128,4.000000e-03
22,22,0.0,7,7.000000e-03,1.000000e+00,0.0,void_Domain::SetupBoundaryConditions(Int_t),512,256,128,7.000000e-03
20,20,0.0,19,1.900000e-02,1.000000e+00,0.0,void_Domain::SetupElementConnectivities(Int_t),512,256,128,1.900000e-02
19,19,0.0,22,2.200000e-02,1.000000e+00,1.0,void_Domain::CreateRegionIndexSets(Int_t_Int_t),512,256,128,2.200000e-02
17,17,0.0,39,3.900000e-02,1.000000e+00,0.0,MPI_Barrier(),512,256,128,3.900000e-02


In [9]:
# ✅関数ごとにfor文を回す
#   * 今回は試験的に１種類の関数で実施
# ✅ExtraP向けの入力データを作成（総実行時間）
# ✅ExtraP向けの入力データを作成（１回実行あたり）
# ✅ExtraPでモデル構築（総実行時間）
# ✅ExtraPでモデル構築（１回実行あたり）
# ✅ExtraPで時間を予測（総実行時間）
# ✅ExtraPで時間を予測（１回実行あたり）
# 関数コール回数予測のためのモデルを構築
# 関数コール回数を予測
# 予測された実行時間と予測されたコール回数をまとめる

functionNames: list[str] = sorted(list(set(trainDF_lulesh["Name"])))
for functionName in functionNames:
    pass

functionName = "Real_t_CalcElemVolume(const"
trainDF_perFunc: pd.DataFrame = trainDF_lulesh[trainDF_lulesh["Name"] == functionName]
testDF_perFunc: pd.DataFrame = testDF_lulesh[testDF_lulesh["Name"] == functionName]

expVar: list[str] = ["process", "iteration", "size"]
resVar: str

In [10]:
# 総実行時間

In [11]:
resVar = "Inclusive"
str_ExtraPinputData: str = gen_ExtraPinputDataFromDF(
    inputDF=trainDF_perFunc,
    expVar=expVar,
    resVar=resVar,
)

filePath: str = f"./extra-p_docker/share/input_lulesh_perFunc.txt"

with open(filePath, mode="w") as f:
    f.write(str_ExtraPinputData)

%sx extrap --text ./extra-p_docker/share/input_lulesh_perFunc.txt  | grep -v Loading | grep -v Measurement

['',
 'Loading file: |          | [00:00<?]',
 'Loading file: 100%|██████████| [00:00<00:00, Creating calltree]',
 'Loading file: 100%|██████████| [00:00<00:00, Validating experiment]',
 'Loading file: 100%|██████████| [00:00<00:00, Validating experiment]',
 '',
 'Generating models: |          | [00:00<?]',
 'Generating models: 100%|██████████| [00:00<00:00]',
 'Generating models: 100%|██████████| [00:00<00:00]',
 'Callpath: reg',
 '\tMetric: time',
 '\t\tModel: 0.008938888890764687 + 0.0003391531590741526 * iteration^(1) + 2.645514630961448e-08 * iteration^(1) * size^(3) * log2(size)^(1)',
 '\t\tRSS: 6.40E+00',
 '\t\tAdjusted R^2: 9.69E-01',
 '']

$$ 総実行時間 =  0.008938888890764687 + 0.0003391531590741526 * iteration + 2.645514630961448 * 10^{-8} * iteration * size^3 * \log_2{size} $$

In [12]:
# 上式より予測対象環境での総実行時間は
time_all = (
    0.008938888890764687
    + 0.0003391531590741526 * test_lulesh_iterations[0]
    + 2.645514630961448e-08
    * test_lulesh_iterations[0]
    * test_lulesh_sizes[0] ** (3)
    * np.log2(test_lulesh_sizes[0])
)
print(time_all)

99.51675178196686


In [13]:
# 1回実行あたり

In [14]:
resVar = "InclusivePerCall"
str_ExtraPinputData: str = gen_ExtraPinputDataFromDF(
    inputDF=trainDF_perFunc,
    expVar=expVar,
    resVar=resVar,
)

filePath: str = f"./extra-p_docker/share/input_lulesh_perFunc.txt"

with open(filePath, mode="w") as f:
    f.write(str_ExtraPinputData)

%sx extrap --text ./extra-p_docker/share/input_lulesh_perFunc.txt | grep -v Loading | grep -v Measurement

['',
 'Loading file: |          | [00:00<?]',
 'Loading file: 100%|██████████| [00:00<00:00, Creating calltree]',
 'Loading file: 100%|██████████| [00:00<00:00, Validating experiment]',
 'Loading file: 100%|██████████| [00:00<00:00, Validating experiment]',
 '',
 'Generating models: |          | [00:00<?]',
 'Generating models: 100%|██████████| [00:00<00:00]',
 'Generating models: 100%|██████████| [00:00<00:00]',
 'Callpath: reg',
 '\tMetric: time',
 '\t\tModel: 1.5456768003451272e-07',
 '\t\tRSS: 4.66E-14',
 '\t\tAdjusted R^2: 1.00E+00',
 '']

$$ 一回当たりの実行時間 = 1.5456768003451272 * 10^{-7} $$

In [15]:
# 上式より、予測対象環境での1回あたりの実行時間は
time_perCall = 1.5456768003451272 * 10 ** (-7)
print(time_perCall)

1.5456768003451272e-07


In [16]:
# 関数コール回数予測のためのモデルを構築

resVar = "#Call"

result_series_list: list[pd.DataFrame] = []

list_modelName: list[str] = [
    "modelIp",
    "modelLog",
    "modelLinAndIp",
    "modelLinAndLog",
    "modelIpAndLin",
    "modelIpAndLog",
    "modelLogAndLin",
    "modelLogAndIp",
    "modelProcessDividedByProblemSize",
    "modelProblemSizeDividedByProcess",
    "modelInfiniteProductOfProblemSizeMultipliedByProcesses",
    "modelInfiniteProductOfProblemSizeDividedByProcesses",
    "modelLinearSumOf2elementCombination",
    "modelLinearSumOfElementCombinations",
    "modelLinearSumOf2elementCombinationWithSquared",
    "modelLinearSumOf2elementCombinationWithCubed",
    "modelSquareRootOfProcess",
    "modelSquareRootTimesOtherElems",
    "modelObeyOneParameter",
    "modelLin"
    # "modelBasicTree",
]

trainDF_perFunc = trainDF_perFunc.rename({"Name": "functionName"}, axis="columns")
testDF_perFunc = testDF_perFunc.rename({"Name": "functionName"}, axis="columns")

models = Models(
    inputDF=trainDF_perFunc,
    expVarColNames=expVar,
    resVarColNames=[resVar],
    targetDF=None,
    modelNames=list_modelName,
)

models.setUpDataBeforeCalcLr()
models.calcLr()
models.calcMAPE()


dictCalcedMAPE = models.returnCalculatedMAPE()
for key in dictCalcedMAPE.keys():
    dictCalcedMAPE[key] = float(dictCalcedMAPE[key])
dict_for_series: dict = copy.deepcopy(dictCalcedMAPE)
dict_for_series["functionName"] = functionName

series: pd.Series = pd.Series(dict_for_series)
result_series_list.append(series)

resultDF: pd.DataFrame = pd.DataFrame(result_series_list)
resultDF = addLowestMAPEColumn(
    inputDF=resultDF, model_name_list=list_modelName, version=2
)
resultDF_after = addLowestMAPEsModelNameColumn(
    inputDF=resultDF, model_name_list=list_modelName, version=2
)


resultDF_after

,modelLin,modelIp,modelLog,modelProcessDividedByProblemSize,modelProblemSizeDividedByProcess,modelInfiniteProductOfProblemSizeDividedByProcesses,modelInfiniteProductOfProblemSizeMultipliedByProcesses,modelLinAndIp,modelLinAndLog,modelIpAndLin,...,modelLinearSumOf2elementCombination,modelLinearSumOfElementCombinations,modelLinearSumOf2elementCombinationWithSquared,modelLinearSumOf2elementCombinationWithCubed,modelSquareRootOfProcess,modelSquareRootTimesOtherElems,modelObeyOneParameter,functionName,最低値,最適モデル
0,1127.007556,1256.690139,1340.469621,108.958047,110.715312,103.503054,100.431976,1256.690139,1340.469621,1127.007556,...,465.142901,178.22039,179.365505,10.878828,1127.007556,950.615455,1127.007556,Real_t_CalcElemVolume(const,10.878828,modelLinearSumOf2elementCombinationWithCubed


In [17]:
bestModelDict: dict = return_bestModelObject(
    inputDF=trainDF_perFunc,
    list_expVar=expVar,
    list_resVar=[resVar],
    list_modelName=list_modelName,
)

bestModel = bestModelDict["object"]

predicted = float(np.array(bestModel.predict(inputDF=testDF_perFunc[expVar])))
_call: float = float(testDF_perFunc.iloc[0][resVar])
_MAPE: float = float(returnMapeScore(l1=[_call], l2=[predicted]))
_series: pd.Series = pd.Series(
    {
        "functionName": functionName,
        "call": _call,
        "MAPE": _MAPE,
        "predicted_call": predicted,
    }
)

call = _call

_series

functionName      Real_t_CalcElemVolume(const
call                              538968000.0
MAPE                                 0.600286
predicted_call               542203350.195741
dtype: object

In [18]:
testDF_perFunc

,index,%Time,Exclusive,Inclusive,#Call,#Subrs,functionName,process,iteration,size,InclusivePerCall
4,4,6.9,1:19.552,79.552,538968000.0,0.0,Real_t_CalcElemVolume(const,512,256,128,1.476006e-07


In [19]:
real_time = testDF_lulesh.loc[0]["Inclusive"]

print(
    f"real_time = {real_time}\ntime_all = {time_all}\ntime_perCall * call= {time_perCall * call}\n"
)

real_time = 1149.925
time_all = 99.51675178196686
time_perCall * call= 83.30703337284125



In [30]:
from sympy import sympify, pprint, symbols

eq1 :str = """0.008938888890764687 + 0.0003391531590741526 * iteration^(1) + 2.645514630961448e-08 * iteration^(1) * size^(3) * log2(size)^(1)"""
eq2 :str = """1.5456768003451272e-07"""

In [32]:
dict_symbols = {}
for elem in expVar:
    dict_symbols[elem] = symbols(elem, real=True)
dict_symbols

{'process': process, 'iteration': iteration, 'size': size}

In [33]:
loaded_eq1 = sympify(eq1, locals=dict_symbols)

loaded_eq2 = sympify(eq2, locals=dict_symbols)


In [34]:
loaded_eq1

2.645514630961448e-8*iteration*size**3*log2(size) + 0.0003391531590741526*iteration + 0.008938888890764687

In [ ]:
ans = loaded_eq1.subs([(dict_symbols["size"], test_lulesh_sizes[0]), (dict_symbols["iteration"], test_lulesh_iterations[0])])
print(type(ans))
float(ans)

14.20299852633616*log2(128) + 0.09576209761374775

In [35]:
loaded_eq2

1.5456768003451272e-7

In [ ]:
%reset

Nothing done.


: 

: 

***

---

___

# 2022年4月17日～

次のような表を作成する

採用される割合 (MAPE の最大値 [%] ，MAPE の最小値 [%]) [%]

| ベンチマークプログラム名 | 線形モデル               | 対数モデル               | 反比例モデル              |
|--------------|---------------------|---------------------|---------------------|
| str          | float(float, float) | float(float, float) | float(float, float) |


目標となるのは一気にこのベンチマークプログラムを作成することだが、既存のライブラリ関数などを利用し、まずはベンチマークごとに作成可能にする。

メモ

## 実装予定

1. 行方向に最小値を検出
2. 最小値以外をNaNに変更
3. 列方向に最小値と最大値を検出

## 


***

---

___